In [1]:
import sims_pars as sims

In [2]:
bn = sims.bayes_net_from_script('''
Pcore SIER {
    b0 ~ norm(12, 1)
    b1 = 0.5 # gggg
    pf ~ beta(8, 20)
    foodstore ~ binom(100, pf)
    b0r ~ norm(0, .01)
    ageA # ~ norm(20, 3)
    ageB ~ norm(30, 2)
    ps ~ beta(5, 6)
    sexA ~ cat({'m': ps, 'f': 1-ps})
    muA = b0 + b0r + b1*ageA
    bmiA ~ norm(muA, sd)
    sdB = sd * 0.5
    muB = b0 + b0r + b1*ageB
    bmiB ~ norm(muB, sdB)
}
''')

In [3]:
bn.Exo

['ageA', 'sd']

In [4]:
sims.sample(bn, cond={'sd': 0.4, 'ageA': 30})

{'sd': 0.4,
 'ageA': 30,
 'b0': 12.187410883222181,
 'b1': 0.5,
 'pf': 0.22660675349832254,
 'b0r': 0.013681751400158471,
 'ageB': 32.10345026631681,
 'ps': 0.289135609724495,
 'foodstore': 27,
 'muA': 27.20109263462234,
 'muB': 28.252817767780748,
 'sexA': 'f',
 'sdB': 0.2,
 'bmiA': 27.37339741818964,
 'bmiB': 28.51929269026648}

In [5]:
root = sims.NodeSet('country')
node_area = root.new_child('area', as_fixed=['b0r', 'ps'], as_floating=['foodstore'])
node_area.new_child('agA', as_fixed=['ageA', 'sexA'], as_floating=['bmiA'])
node_area.new_child('agB', as_fixed=['ageB'], as_floating=['bmiB'])

sc = sims.as_simulation_core(bn, root)

root.print()
root.print_samplers()

NodeSet country
|-Fixed ['b1', 'ageA', 'sd']
|-NodeSet area
  |-Fixed ['b0r', 'ps']
  |-Floating ['foodstore', 'pf']
  |-NodeSet agA
    |-Fixed ['ageA', 'sexA']
    |-Floating ['b0', 'bmiA', 'muA']
    |-Exo ['ps']
    |-Listening ['sd', 'b1', 'b0r']
  |-NodeSet agB
    |-Fixed ['ageB', 'sdB']
    |-Floating ['b0', 'muB', 'bmiB']
    |-Listening ['sd', 'b1', 'b0r']
NodeSet country
|-Cp: muB, (b0, b0r, ageB)| (b1)
|-Cp: bmiB, (b0, b0r, ageB, muB, sdB)| (b1, sd)
|-Cp: muA, (b0, b0r)| (b1, ageA)
|-Cp: foodstore, (pf)
|-Cp: bmiA, (b0, b0r, muA)| (b1, ageA, sd)
|-NodeSet area
  |-Cp: muB, (b0, ageB)| (b1, b0r)
  |-Cp: bmiB, (b0, ageB, muB, sdB)| (b1, b0r, sd)
  |-Cp: muA, (b0)| (b1, b0r, ageA)
  |-Cp: foodstore, (pf)
  |-Cp: bmiA, (b0, muA)| (b1, b0r, ageA, sd)
  |-Fz: pf
  |-NodeSet agA
    |-Fz: b0
    |-Cp: bmiA, (b0, muA)| (b1, b0r, ageA, sd)
    |-Cp: muA, (b0)| (b1, b0r, ageA)
  |-NodeSet agB
    |-Fz: b0
    |-Cp: muB, (b0)| (b1, b0r, ageB)
    |-Cp: bmiB, (b0, muB)| (b1, b0r, ageB,

In [6]:
bn.RVRoots

['b0', 'pf', 'b0r', 'ageB', 'ps']

In [7]:
sc

Simulation core: SIER